In [8]:
import numpy as np
import numpy.typing as npt

import sys
from pathlib import Path
# adjust '..' to point to your repository root that contains the SpiderSolitaire package
sys.path.insert(0, str(Path('..').resolve()))
from env.spider_space import SpiderSpace
from env.spider_env import SpiderEnv

In [ ]:
depths = np.array([1,3,0,1, 4])

__N_PILES = 5#SpiderSpace.N_PILES #5
__MAX_SEQ_LEN = 4#SpiderSpace.HIGHEST_RANK-1 #4
src = np.arange(0, __N_PILES*__N_PILES * __MAX_SEQ_LEN, __N_PILES*__MAX_SEQ_LEN)
pos = src+depths 
pos = pos[pos!=src]#excluding 0-depth destinations
actions = np.arange(__N_PILES*__N_PILES * __MAX_SEQ_LEN)#.reshape((5,5))
# mask = (actions%5==src)
pos


piles = np.array([[1,4,5,6,8],
                 [15,11,9,0,0],
                  [0,0,0,0,0],
                 [7,0,0,0,0],
                 [7,6,0,0,0],
                 [10,11,19,8,0]])

depths = SpiderSpace._get_max_sequences_depths(piles)
print(f"Calc depths: {depths}")
s = np.arange(__N_PILES)#src//(__N_PILES * __MAX_SEQ_LEN)
top_idc = SpiderSpace._get_top_cards_indices(piles)
print(f"Top cards: {piles[top_idc]}")

#if taking max depths:
cols= np.arange(piles.shape[0])
mask = (cols[depths!=0] - depths[depths!=0][:,None])<0
print("mask: ",mask)

print(f"{(depths[depths!=0][:,None]*mask)}")

#print(f"masked depths:\n{ depths[depths!=0][:,None][mask]}")
# if max depth is taken:
src_trgt_diff = piles[top_idc[0], top_idc[1]-depths[depths!=0  ]+1][:,None] - piles[top_idc]
#if 1 depth is taken:
src_trgt_diff = piles[top_idc][:,None] - piles[top_idc]

print(f"src trgt diff:\n {src_trgt_diff}")
src, trgt = np.nonzero(src_trgt_diff ==-1)
# src -> trgt
print("Src piles and targets: ", top_idc[0][src], top_idc[0][trgt] )

#source*__N_PILES * __MAX_SEQ_LEN + target * __MAX_SEQ_LEN + depth-1
# or,, if depths is const 1:
src*__N_PILES  + trgt 


###################################


first_cards_pos = top_idc[1] - depths[top_idc[0]]
first_cards_pos

import numpy as np
depths = np.array([1,3,0,1, 4])

__N_PILES = 5
__MAX_SEQ_LEN = 4

# tableau_piles, stock_pile, counts, depths = self._unpack_state(self._internal_state)
def mask_moves(tableau_piles=None, counts: npt.NDArray[np.integer]=None, depths: npt.NDArray[np.integer]=None, 
               _maxheight:int=64,
               __N_PILES=SpiderSpace.N_PILES,
              __N_MOVES = SpiderSpace.N_PILES * SpiderSpace.N_PILES * (SpiderSpace.HIGHEST_RANK-1) +1,
            ) -> npt.NDArray[np.bool]:
    """"""
    actions_mask = np.zeros(__N_MOVES, dtype=bool)#.reshape((5,5))

    #step 0: calculate which cards can be moved from nonemtpy piles to nonempty piles
    if tableau_piles is None:
        tableau_piles = np.array([[1,4,5,6,8],
                 [15,11,9,0,0],
                 [0,0,0,0,0],
                 [7,0,0,0,0],
                 [7,6,0,0,0],
                 [10,11,19,8,0],
                 [0,0,0,0,0]
                                 ])
    if counts is None:
        counts = SpiderSpace._get_facedown_counts(tableau_piles)
    if depths is None:
        depths = SpiderSpace._get_max_sequences_depths(piles)
    
    print(f"Calc depths: {depths}")

    #top cards indices per eaxh pile [empty piles are omitted]
    top_idc = SpiderSpace._get_top_cards_indices(piles)
    print(f"Top cards: {piles[top_idc]}")

    ##if taking max depths:
    #cols= np.arange(piles.shape[0])
    # masks which cards can be moved:
    mask = (cols[depths!=0] - depths[depths!=0][:,None])<0
    print("mask: ",mask)

    #print(f"{(depths[depths!=0][None,:]*mask)}")
    acc_depths = mask.cumsum(axis=1)#mask[:, ::-1].cumsum(axis=1)[:,::-1]
    max_depth = acc_depths.max()
    acc_depths = acc_depths[:,:max_depth]
    # (N nonepty piles, max seq len)
    print(f"acc depths: {acc_depths}")
    print(f"piles as is: {tableau_piles}" )

    depth_piles = tableau_piles[top_idc[0]]* mask[None,:].T[:max_depth] # each pile per depth
    # e.g. all the piles are present at the first coordinate
    # thepiles with a top sequencs of depth>=2 are present at the second coordinate,
    # etc
    print("depth piles",depth_piles)
    print("top idc", top_idc[1]) 
    idc_base_seq_cards = top_idc[1][:,None]-acc_depths+1

    print(idc_base_seq_cards)

    depth_ax, pile_ax = np.arange(max_depth)[None,:], np.arange(depth_piles.shape[1])[:,None]
    base_seq_cards = depth_piles[depth_ax, pile_ax, idc_base_seq_cards].T #- piles[top_idc]
    print(f"base seq cards:\n {base_seq_cards}")

    #src_trgt_diff= base_seq_cards[:,:,None] - base_seq_cards[:,None]
    src_trgt_diff =tableau_piles[top_idc][:,None] - base_seq_cards[:,None]
    #depth_piles[depth_ax, pile_ax] - 

    print(src_trgt_diff)
    idepths, trgt, src = np.nonzero(src_trgt_diff ==1)
    # src -> trgt
    print("depths-1, targets, src piles: ", idepths, trgt, src)#top_idc[0][src], top_idc[0][trgt], idepth )

    # calcularing actions indices
    # converting to original cooords:

    msk = ((src!= trgt) & #check whether not same src and trgt
           (top_idc[1][trgt]+idepths+1<_maxheight) #check whether max pile height will not be exceeded
          )
    valid_moves_idc = SpiderEnv._flatten_move_action(top_idc[0][src[msk]], 
                                                     top_idc[0][trgt[msk]], 
                                                     (idepths+1)[msk])
    #source*__N_PILES * __MAX_SEQ_LEN + target * __MAX_SEQ_LEN + depth-1

    actions_mask[valid_moves_idc] = True

    
    # step2: True if possible to move cards from nonemtpy piles to empty piles:
    if top_idc[0].shape[0]< tableau_piles.shape[0]:
        empty_piles_mask = np.ones(tableau_piles.shape[0], 
                               dtype=bool)
        empty_piles_mask[top_idc[0]] = False
        empty_piles_idc = np.nonzero(empty_piles_mask)[0]
        #print("empty piles idc", empty_piles_idc)
        #src_trgt_diff0 = tableau_piles[empty_piles_idc][:, None] - base_seq_cards[:, None]
        #print("empty piles <- seqs diffs", src_trgt_diff0)
    
        #idepths0, trgt0, src0 = np.nonzero(src_trgt_diff0<0)
        idepths0, src0 = np.nonzero(base_seq_cards)
        n_src0, n_trgt0 = src0.shape[0], empty_piles_idc.shape[0]
        idepths0, orig_src0 = np.tile(idepths0, n_trgt0), np.tile(top_idc[0][src0], n_trgt0)
        trgt0 = np.repeat(empty_piles_idc, n_src0)
        print(trgt0, idepths0, orig_src0)
        #print(idepths0, empty_piles_idc[trgt0], top_idc[0][src0])
        valid_moves0_idc = SpiderEnv._flatten_move_action(orig_src0,
                                                     trgt0,
                                                     idepths0+1)

        actions_mask[valid_moves0_idc] = True

    else:#step 3: calculate whether it is possible to deal cards [it's the last move] 
        actions_mask[-1] = (counts[-1]>0) and np.all(top_idc[0]<_maxheight-1)
    
    print("idc of valid moves: ", np.nonzero(actions_mask))
        
    return actions_mask

mask_moves()

Calc depths: [1 1 0 1 2 1]
Top cards: [8 9 7 6 8]
mask:  [[ True False False False False]
 [ True False False False False]
 [ True False False False False]
 [ True  True False False False]
 [ True False False False False]]
[[1 0 0 0 0]
 [1 0 0 0 0]
 [1 0 0 0 0]
 [2 2 0 0 0]
 [1 0 0 0 0]]
src trgt diff:
 [[ 0 -1  1  2  0]
 [ 1  0  2  3  1]
 [-1 -2  0  1 -1]
 [-2 -3 -1  0 -2]
 [ 0 -1  1  2  0]]
Src piles and targets:  [0 3 3 4 5] [1 0 5 3 1]
Calc depths: [1 1 0 1 2 1]
Top cards: [8 9 7 6 8]
mask:  [[ True False False False False]
 [ True False False False False]
 [ True False False False False]
 [ True  True False False False]
 [ True False False False False]]
acc depths: [[1 1]
 [1 1]
 [1 1]
 [1 2]
 [1 1]]
piles as is: [[ 1  4  5  6  8]
 [15 11  9  0  0]
 [ 0  0  0  0  0]
 [ 7  0  0  0  0]
 [ 7  6  0  0  0]
 [10 11 19  8  0]
 [ 0  0  0  0  0]]
depth piles [[[ 1  4  5  6  8]
  [15 11  9  0  0]
  [ 7  0  0  0  0]
  [ 7  6  0  0  0]
  [10 11 19  8  0]]

 [[ 0  0  0  0  0]
  [ 0  0  0  0  0

array([False, False, False, ..., False, False, False], shape=(1201,))

In [10]:
 # SpiderSpace._left_align_rows(o)

In [ ]:
#SpiderSpace._get_max_sequences_depths(SpiderSpace._vectorize(e._internal_state, _counts_features=False))

In [ ]:
# ee._internal_state[4,16]=0

#ee._internal_state[-1,:10] = SpiderSpace._get_max_sequences_depths(ee._internal_state[:10])
#ee._internal_state
#e.discard_complete_sequences()

In [ ]:
env = SpiderEnv(vectorize_obs=True, n_suits=1, _diagnostics_mode=0.8 )
obs = env.reset()

n_test=1000

env.random_play(n_max_iterations=n_test, _render_state_timeout=0, verbosity=3,  _diagnostic_mode=True)
#print(env._internal_state)


Avg reward per valid action: -0.1911911911911889
Number of complete sequences: 0
Stock cards: 0
🂠 🂠 🂠 🂠 🂠 🂠 🂠 🂠 🂠 🂠
🂠 🂠 🂠 🂠 🂠 🂠 🂠 🂠 🂠 🂠
🂠 🂠 🂠 🂠 🂠 🂠 🂠 🂠 🂠 🂠
🂠 🂠 🂠 🂠 🂦 🂠 🂠 🂢 🂠 🂠
🂠 🂠 🂠 🂠 🂣 🂦 🂭   🂨 🂡
🂥 🂣 🂡 🂩 🂮 🂥 🂨   🂧 🂢
🂩 🂫 🂫 🂭 🂮 🂧 🂡   🂦 🂤
🂤 🂢 🂣 🂤 🂭 🂦 🂮   🂨 🂥
🂦 🂡 🂢 🂢   🂩 🂭   🂧 🂧
🂥     🂡   🂨 🂫   🂪 🂦
🂨         🂢 🂪   🂤 🂥
🂥         🂧 🂩   🂣  
🂤               🂥  
🂣                  
🂢                  
🂡                  
Face-down cards counts: [5 5 5 5 3 4 4 3 4 4 0]
Depths of top sequences: [5 2 2 2 2 1 5 1 1 3]
Cards in stock: 


False

In [14]:
env = SpiderEnv(vectorize_obs=True)
obs = env.reset()

cum_rew = 0
n_test=5#0000
for i in range(n_test):
    a = env.action_space.sample()
    masked_actions = env.get_action_mask()

    if i%100==0:
        act_idc = np.nonzero(masked_actions)[0]
        print(f"available actions: {[env._unflatten_move_action(_a) for _a in act_idc if _a!=1200], 1200 if 1200 in act_idc else -1}")
        env.render()
        
    res =  env._num2act(a)
    if  res!=masked_actions[a]:
        print(f"Chosen action: {a}=={env._unflatten_move_action(a)}")
        if res:
            env.render()


print(f"Avg reward per act: {cum_rew/n_test}")
env.render(print_auxillary_info=True)#, show_agents_obs=True)

available actions: ([(2, 3, 1), (4, 2, 1), (5, 9, 1), (6, 9, 1), (7, 2, 1), (8, 0, 1), (9, 1, 1), (10, 0, 2)], 1200)
Number of complete sequences: 0
Stock cards: 50
🂠 🂠 🂠 🂠 🂠 🂠 🂠 🂠 🂠 🂠
🂠 🂠 🂠 🂠 🂠 🂠 🂠 🂠 🂠 🂠
🂠 🂠 🂠 🂠 🂠 🂠 🂠 🂠 🂠 🂠
🂠 🂠 🂠 🂠 🂠 🂠 🂠 🂠 🂠 🂠
🂠 🂠 🂠 🂠 🃄 🃊 🂪 🂴 🂡 🃛
🃒 🂽 🃅 🃖            
Avg reward per act: 0.0
Number of complete sequences: 0
Stock cards: 50
🂠 🂠 🂠 🂠 🂠 🂠 🂠 🂠 🂠 🂠
🂠 🂠 🂠 🂠 🂠 🂠 🂠 🂠 🂠 🂠
🂠 🂠 🂠 🂠 🂠 🂠 🂠 🂠 🂠 🂠
🂠 🂠 🂠 🂠 🂠 🂠 🂠 🂠 🂠 🂠
🂠 🂠 🂠 🂠 🃄 🃊 🂪 🂴 🂡 🃛
🃒 🂽 🃅 🃖            
Face-down cards counts: [ 5  5  5  5  4  4  4  4  4  4 50]
Depths of top sequences: [1 1 1 1 1 1 1 1 1 1]
Cards in stock: 🂳 🃔 🂫 🂻 🃉 🃊 🃝 🂣 🂵 🃆 🂥 🂶 🃂 🃞 🃁 🃛 🃚 🂷 🂵 🃗 🂳 🂥 🃓 🂸 🃝 🂷 🃕 🃓 🂦 🂭 🃑 🂱 🂧 🂦 🃇 🃑 🃗 🃂 🂱 🂢 🃁 🂫 🂣 🂶 🂸 🂺 🃇 🂭 🃍 🂨
